In [45]:
import os
import torch
from openai import OpenAI
from transformers import GPT2Tokenizer
from dotenv import load_dotenv

In [46]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

In [7]:
#RQ1: Step 1 - Setting up a prior belief based on targeted question

#targeted questions
question1 = "Do you prefer direct next steps or detailed discussion first?"
question1_options=["1. direct steps", "2. detailed discussion"]

question2= "Would you prefer a checklist or conversation?"
question2_options=["1. checklist", "2. conversation"]

In [8]:
#Function for asking question

def ask_question(question, answer):
    #question 1
    print(question)
    for i in range(len(answer)):
        print(answer[i])
    choice= input('Enter your preferred answer number') #change it to better UI based button later
    return answer[int(choice)-1]


choice_result = []

choice_result.append(ask_question(question1, question1_options))
choice_result.append(ask_question(question2, question2_options))

print(choice_result)


Do you prefer direct next steps or detailed discussion first?
1. direct steps
2. detailed discussion
Would you prefer a checklist or conversation?
1. checklist
2. conversation
['1. direct steps', '1. checklist']


In [23]:
#creating user profile based on user's answer

user_profile = {"preferred_conversation_style": None,
                "ph1": None,
                "ph2": None,
                "peh1": None,
                "peh2": None}

#take 2 answers and based on that assign key-value pair for user profile--think


if choice_result[0]=="1. direct steps" and choice_result[1] == "1. checklist":
    user_profile["preferred_conversation_style"] = "action_based"
elif choice_result[0]=="2. detailed discussion" and choice_result[1] == "2. conversation":
    user_profile["preferred_conversation_style"] = "relationship_based"
else:
    user_profile["preferred_conversation_style"] = "mixed"
        
print(user_profile)


{'preferred_conversation_style': 'action_based', 'ph1': None, 'ph2': None, 'peh1': None, 'peh2': None}


In [30]:
#assign prior belief value - hardcoded, might change with an LLM prompt layer (step 1 completed)

if user_profile["preferred_conversation_style"] == "action_based":
    user_profile["ph1"] = .90
    user_profile["ph2"] = .10
elif user_profile["preferred_conversation_style"] == "relationship_based":
    user_profile["ph1"] = .10
    user_profile["ph2"] = .90
else:
    user_profile["ph1"]=.60
    user_profile["ph2"]=.40
    
print(user_profile)

{'preferred_conversation_style': 'action_based', 'ph1': 0.9, 'ph2': 0.1, 'peh1': 0.4, 'peh2': 0.6}


In [11]:
#RQ1: Step 2 - Observing behavior with LLM prompt and treating each message like evidance P(E)

user_input = "Can you elaborate this topic?"                          #input("Hi there! What you want to chat about?")

In [ ]:
#torch.set_default_device("cuda")


In [ ]:
#model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
#tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.56s/it]


In [ ]:

#tokenizer.pad_token = tokenizer.eos_token

# Create full prompt with few-shot examples
full_prompt = f'''
You are a conversation analyst. Estimate two probabilities:
- peh1 = probability (0 to 1) that the user is Action-Based
- peh2 = probability (0 to 1) that the user is Relationship-Based

Here are some examples:

Example 1:
User: "Tell me the fastest way to finish this project."
peh1: 0.90
peh2: 0.10

Example 2:
User: "Before we start, can we discuss what's most important to focus on together?"
peh1: 0.20
peh2: 0.80

Example 3:
User: "I just want to get this over with quickly."
peh1: 0.85
peh2: 0.15

Now, analyze the following user input:

User: {user_input}

Output:
peh1:
peh2:
'''

# Send the full prompt to OpenAI
response = client.chat.completions.create(
    model="gpt-4o-mini",  # or "gpt-3.5-turbo", or "gpt-4o-mini"
    messages=[
        {"role": "user", 
         "content": full_prompt},
    ]
)

# Print the response text
response_text = response.choices[0].message.content
print(response_text)


To analyze the provided user input, "Can you elaborate this topic?", we need to assess the user's intent, focusing on whether they are more action-based or relationship-based.

An action-based user usually focuses on efficiency, outcomes, or getting tasks done quickly. They might use language that is direct and oriented towards completing a specific task or objective.

A relationship-based user tends to focus on collaboration, understanding, and shared goals. They often value communication and mutual agreement.

In this instance, the user's request for elaboration does not strongly indicate a focus on quick results or specific actions. Instead, it suggests a desire for more information and deeper understanding, which can align more with a collaborative and communicative approach.

Given this analysis, an estimate for the probabilities could be:

Output:
peh1: 0.40
peh2: 0.60


In [31]:
for lines in response_text.splitlines():
    if lines.strip().startswith("peh1"):
        user_profile["peh1"] = float(lines.split(":")[1].strip())
    elif lines.strip().startswith("peh2"):
        user_profile["peh2"] = float(lines.split(":")[1].strip())
        
print(user_profile)

{'preferred_conversation_style': 'action_based', 'ph1': 0.9, 'ph2': 0.1, 'peh1': 0.4, 'peh2': 0.6}


In [39]:
#using bayesian theorem, calculating posterior 

numerator = user_profile["peh1"] * user_profile["ph1"]
denominator = (user_profile["peh1"] * user_profile["ph1"]) + (user_profile["peh2"] * user_profile["ph2"])

ph1e = numerator/denominator

ph2e = 1 - ph1e

print(ph1e, ph2e)

# after first input
user_profile["ph1"] = ph1e
user_profile["ph2"] = ph2e

0.3450134770889489 0.6549865229110511


In [43]:
# Create full prompt with few-shot examples
updated_prompt = f'''
You're a Computer Science note maker. 
Generate notes for the user according to their preferred conversation style based on {ph1e} and {ph2e} score.

ph1e means probability the user is action based given what they just said.
ph2e means probability the user is relationship based given what they just said.

if the user is action based, give output in the following format -
1. direct and simple sentence
2. short and concise sentence
3. task focused and in bullet points

if the user is relationship based, give output in the following format -
1. direct and simple sentence
2. storyline conversation
3. focus on emotion and in paragraph style

Now generate notes on Array using python coding language in 300 words. 
'''

# Send the full prompt to OpenAI
response = client.chat.completions.create(
    model="gpt-4o",  # or "gpt-3.5-turbo", or "gpt-4o-mini"
    messages=[
        {"role": "user", "content": updated_prompt},
    ]
)

# Print the response text
response_text = response.choices[0].message.content
print(response_text)


### Notes on Arrays in Python

#### Action-Based Style

1. Create an array using Python's list feature.
2. Access elements with index positions starting from 0.
3. Example: `my_array = [1, 2, 3, 4, 5]`.
4. Use `append()` to add an element: `my_array.append(6)`.
5. Retrieve value: `value = my_array[2]`.
6. Length of array: `len(my_array)`.
7. Use loops to iterate over array elements.
8. Remove element with `remove()`: `my_array.remove(3)`.
9. Array slicing example: `sub_array = my_array[1:4]`.
10. Sort array: `my_array.sort()`.

#### Relationship-Based Style

Arrays in Python are vital, offering a way to handle collections of data. Imagine a situation where you have a list of friends you want to manage smoothly. In Python, this is where arrays come in handy.

Start by creating an array, much like writing down names on paper, using Python's intuitive list feature. For instance, you can jot down: `friends_list = ["Alice", "Bob", "Charlie"]`. Each item is at a specific index, emotions tagg

In [44]:
#without bayesian theorem approach 
without_theorem_prompt = f'''
You're a Computer Science note maker. 

Now generate notes on Array using python coding language in 300 words. 
'''

# Send the full prompt to OpenAI
response = client.chat.completions.create(
    model="gpt-4o",  # or "gpt-3.5-turbo", or "gpt-4o-mini"
    messages=[
        {"role": "user", "content": without_theorem_prompt},
    ]
)

# Print the response text
response_text = response.choices[0].message.content
print(response_text)



# Arrays in Python

### Introduction
An array is a data structure that stores a collection of elements, usually of the same data type, in a contiguous block of memory. Arrays allow efficient access and manipulation of elements using indices.

### Creating Arrays
Python doesn't have a built-in array data type like some other languages (e.g., C, Java). However, you can use two main approaches for working with arrays:

1. **Lists**: The most common Python data structure that can be used as an array. Lists are flexible but not as efficient in terms of speed and memory as true arrays.
   ```python
   # Creating a list
   my_list = [1, 2, 3, 4, 5]
   ```

2. **Array Module**: Provides a true array data structure but is limited to basic types and operations.
   ```python
   import array
   # Creating an array of integers
   my_array = array.array('i', [1, 2, 3, 4, 5])
   ```

3. **NumPy Arrays**: NumPy is a powerful library for numerical computing in Python. NumPy arrays provide more function